In [5]:
import numpy as np
import cv2


In [3]:
from ultralytics import YOLO

In [4]:
import cvzone
import math
model=YOLO("yolov8n.pt")

In [1]:
from sort import *

C:\Users\yash\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [6]:
import cvzone
import math
model=YOLO("yolov8n.pt")
classNames =["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", 
             "fire nydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", 
             "cow", "elephant", "bear", "Zebra", "giraffe", "backpack", "umbrella", "handbag" "tie", 
             "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "haseball bat",
             
             "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass",
             "cup", "fork", "knife", "spoon", "bowt", 
             "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
             "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", 
             "diningtable", "toilet", "tvmonitor", "Laptop", "mouse", 
             "remote", "keyboard", "cell phone", "cellphone", "oven", 
             "toaster", "sink", "refrigeraton", "book", "clock", "vase", 
             "scissors", "teddy bear", "hair drier","toothbrush"]


tracker=Sort(max_age=20,min_hits=3,iou_threshold=0.3)
limits=[723,500,1473,500]
totalCount=[]

cap=cv2.VideoCapture('13.mp4')
cap.set(3,640)
cap.set(4,480)
mask=cv2.imread("mask_car.png")
while True:
    success,img=cap.read()
    mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))  # Match width and height of img
    imgRegion = cv2.bitwise_and(img, mask_resized)

#     imgRegion=cv2.bitwise_and(img,mask)
    results=model(imgRegion,stream=True)
    detections=np.empty((0,5))
    for r in results:
        boxes=r.boxes
        for box in boxes:
            #bounding box
            
            x1,y1,x2,y2=box.xyxy[0]

            x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
#             cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
            w,h=x2-x1,y2-y1
#             bbox=int(x1),int(y1),int(w),int(h)
            
            
            
            print(x1,y1,x2,y2)

            cvzone.cornerRect(img,(x1,y1,w,h),l=15,colorR=(0, 0, 255),colorC=(0, 255, 255))
#             confidence
            conf=math.ceil((box.conf[0]*100))/100
                            
            print(conf)
            cls=int(box.cls[0])
            currentclass=classNames[cls]
            if currentclass=='car'or currentclass=='bus'or currentclass=='truck' or currentclass=='motorbiker' and conf>0.3:
                
#                 cvzone.putTextRect(img,f'{classNames[cls]} {conf}',(max(0,x1),max(15,y1)),scale=1,thickness=1,colorT=(0, 0, 255),
#                                colorR=(255, 0,0),offset=3)
                cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=1,colorR=(0, 0, 255),colorC=(0, 255, 255))
                currentArray=np.array([x1,y1,x2,y2,conf])
                detections=np.vstack((detections,currentArray))
                
                
    resultTracker=tracker.update(detections)
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,250),thickness=5)
    for result in resultTracker:
        x1,y1,x2,y2,id=result
        x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h=x2-x1,y2-y1
        
        cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=1,colorR=(100, 255, 255),colorC=(0, 150, 255))
        cvzone.putTextRect(img,f'{classNames[cls]} {conf}',(max(0,x1),max(35,y1)),scale=1,thickness=2,colorT=(0, 0, 255),
                               colorR=(255, 0,0),offset=3)
        cx,cy=x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
        
        if limits[0]<cx<limits[2] and limits[1]-20<cy<limits[1]+20:
            if totalCount.count(id)==0:
                
            
                totalCount.append(id)
    cvzone.putTextRect(img,f'COUNT:{len(totalCount)}',(50,50),scale=1,thickness=2,colorT=(0, 0, 255),
                               colorR=(255, 0,0),offset=3)
        
    cv2.imshow("images",img)
#     cv2.imshow("imagesRegion",imgRegion)
    cv2.waitKey(1)
        # Wait for 1ms, check if 'q' key is pressed
    key = cv2.waitKey(1)
    if key == 27:  # 'q' key to quit
        break


cap.release()
cv2.destroyAllWindows()



0: 384x640 6 persons, 10 cars, 2 trucks, 136.5ms
836 504 918 586
0.88
0 686 349 880
0.82
28 568 259 700
0.8
1446 659 1492 780
0.73
1511 669 1556 786
0.72
523 535 694 627
0.7
1426 522 1460 607
0.67
752 449 834 515
0.64
1278 500 1308 581
0.54
239 587 446 708
0.52
1596 668 1634 745
0.51
465 640 635 773
0.47
420 519 525 606
0.43
1850 512 1919 591
0.39
592 406 624 459
0.34
961 434 1001 495
0.31
563 484 645 538
0.3
1017 463 1068 532
0.27
Speed: 5.0ms preprocess, 136.5ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)
[       1017         463        1068         532          12]
[        563         484         645         538          11]
[        961         434        1001         495          10]
[       1850         512        1919         591           9]
[        420         519         525         606           8]
[        465         640         635         773           7]
[        239         587         446         708           6]
[        752         449       

[      953.6      439.21       999.7       500.3          10]
[     1850.3       510.6      1919.2      591.31           9]
[     379.81       659.9      572.64       805.3           7]
[     142.49      600.08      376.56      739.74           6]
[     733.29      455.79      821.21      523.25           5]
[     469.36      545.93      648.63      642.01           4]
[      -7.93      585.38      199.39      718.51           3]
[    -8.7693      724.41       259.7      914.83           2]
[     826.56      515.56      913.43      601.47           1]

0: 384x640 8 persons, 11 cars, 2 trucks, 1 boat, 83.9ms
827 516 913 603
0.89
0 728 234 919
0.79
464 548 645 644
0.78
1508 653 1551 775
0.75
732 457 818 524
0.74
0 583 186 727
0.7
124 603 365 751
0.69
1444 650 1490 767
0.67
1429 523 1465 608
0.55
1277 501 1304 576
0.47
365 662 563 818
0.42
1496 631 1551 777
0.41
1607 669 1656 742
0.38
1852 510 1919 592
0.34
951 443 998 502
0.33
1483 550 1785 669
0.33
1030 423 1091 468
0.32
1430 495 1469 6

[    -4.6739      750.25      140.92      894.61          22]
[     286.88      684.11      476.37      835.72          21]
[    -1.8626       601.4      127.92      725.58          20]
[     1034.1       421.5      1089.1      467.11          18]
[     1010.2       478.6      1065.2      554.78          12]
[     947.55      444.72         995      505.33          10]
[     1852.2      511.38      1919.1       590.7           9]
[       46.5       618.2      303.27      779.14           6]
[      714.7         463      807.45         533           5]
[      409.9      558.12      604.74      658.62           4]
[     817.58      528.19      909.31      617.33           1]

0: 384x640 6 persons, 11 cars, 3 trucks, 96.5ms
815 530 910 620
0.79
1504 644 1550 769
0.74
711 465 805 535
0.74
398 559 602 658
0.7
36 622 290 786
0.7
1439 646 1488 760
0.61
1429 525 1464 610
0.58
0 752 117 906
0.57
324 535 435 635
0.46
274 687 463 838
0.43
0 600 116 724
0.42
1611 673 1660 755
0.42
1034 416 1088 46

[     1037.5      418.46      1090.6       466.8          18]
[     947.89      450.08      994.22      511.07          10]
[      284.9      542.78      402.27      643.61           8]
[     182.15      709.47      386.99      877.78           7]
[     696.81       473.4      794.12      545.64           5]
[      352.8      571.17      562.78       676.9           4]
[     807.71       541.5      907.52      639.04           1]

0: 384x640 7 persons, 13 cars, 1 truck, 1 boat, 81.8ms
694 475 792 547
0.81
341 570 557 679
0.75
1507 644 1550 764
0.72
805 543 906 641
0.69
167 711 373 882
0.6
1438 532 1470 613
0.6
1444 646 1492 750
0.53
0 642 215 819
0.53
1621 675 1659 755
0.51
276 545 395 643
0.47
883 441 932 500
0.38
1038 569 1110 677
0.38
1011 488 1074 568
0.38
552 421 581 467
0.35
0 644 217 819
0.35
791 435 858 493
0.35
1490 543 1794 664
0.34
950 450 995 512
0.33
1036 420 1090 468
0.31
1478 892 1532 937
0.3
920 415 968 462
0.26
1281 489 1309 570
0.26
Speed: 2.0ms preprocess, 81.8ms inf

0: 384x640 9 persons, 10 cars, 4 trucks, 2 boats, 80.8ms
671 480 779 561
0.76
284 588 509 698
0.66
1449 637 1495 751
0.64
1448 532 1483 615
0.57
1039 420 1099 471
0.55
1511 641 1569 751
0.52
1017 500 1073 577
0.5
953 456 997 520
0.49
1481 879 1538 933
0.48
798 561 906 662
0.46
1629 675 1684 753
0.42
1510 645 1552 754
0.41
0 672 129 822
0.4
51 743 278 900
0.38
228 557 351 664
0.36
226 559 354 665
0.33
0 675 131 827
0.31
284 584 510 697
0.29
1803 681 1919 842
0.29
878 452 930 506
0.29
7 511 29 574
0.28
0 498 27 574
0.28
1287 482 1314 567
0.27
1498 542 1802 660
0.27
400 526 508 604
0.26
Speed: 2.0ms preprocess, 80.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[    -6.1845       674.4      136.48      821.25          25]
[     876.01      447.39      930.85      506.58          24]
[     1041.1      420.97      1097.9       469.5          18]
[     1015.9      499.66      1073.6      577.68          12]
[     952.09      455.54      997.17       518.8          10]
[ 

...

[     2.6929      684.46      194.33      849.94         272]
[     638.23      513.53      745.24      596.18         265]
[     1360.6      487.26      1459.4      551.41         263]
[     1010.4      456.23        1067      516.97         255]
[     1235.4         457      1292.7      516.08         248]
[      829.1      484.15      928.88      567.97         245]
[     585.11      451.67       630.3      509.01         238]
[     797.97      465.07      871.37      525.62         237]
[     234.89      601.26      368.19      703.28         227]
[     539.92       596.9      663.19      710.28         221]
[     1362.7      487.35      1521.4      569.59         199]
[     251.74      461.47      344.39         517         184]

0: 384x640 4 persons, 20 cars, 2 motorcycles, 3 trucks, 2 boats, 99.5ms
1592 754 1645 895
0.74
1115 824 1185 938
0.7
1010 455 1068 518
0.65
1456 509 1532 549
0.57
856 832 988 937
0.57
1474 557 1505 638
0.56
1237 457 1294 515
0.53
635 512 746 595
0.5
1362 

0.36
169 608 314 721
0.34
1561 535 1872 656
0.32
606 525 703 606
0.32
858 715 967 867
0.31
777 464 863 533
0.3
1852 532 1919 615
0.27
342 455 391 496
0.26
Speed: 3.2ms preprocess, 94.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
[      863.3      596.99      957.29       713.5         278]
[     610.78      519.25      740.57      605.63         265]
[    0.51619       375.4      137.64      516.51         262]
[     1006.1      457.85      1064.5      521.81         255]
[     1243.8      455.63      1300.6      512.03         248]
[     801.34      486.98      916.81      575.92         245]
[    -11.251      689.02      137.47      828.19         239]
[     779.42      465.24       860.3      532.01         237]
[     858.55      716.39      966.45       866.2         234]
[     1385.5      482.45      1552.7      554.29         199]
[     247.67      457.14      355.88      519.05         184]
[     389.62      438.46       493.1      498.54         183]

0: 


0: 384x640 2 persons, 13 cars, 1 motorcycle, 1 truck, 81.7ms
1597 762 1661 908
0.8
565 522 714 616
0.74
999 460 1064 527
0.7
770 495 891 584
0.66
1419 481 1581 561
0.64
199 702 307 826
0.59
1332 527 1410 606
0.58
419 621 566 763
0.53
1481 545 1512 632
0.53
1050 527 1115 616
0.46
90 620 249 743
0.46
830 753 948 920
0.44
849 619 944 750
0.41
592 432 636 486
0.41
756 462 850 533
0.36
262 450 388 515
0.34
1067 412 1125 461
0.27
Speed: 3.0ms preprocess, 81.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
[     1331.9      525.94      1410.9      607.04         280]
[     848.11      620.32      943.75      743.06         278]
[     565.82       522.3      713.45      615.73         265]
[     999.36      460.91      1063.6      526.78         255]
[     771.15      494.78      889.64      585.01         245]
[     591.76      433.09       635.8      485.09         238]
[     758.13       463.8      845.94      533.98         237]
[     92.389      621.58      247.49    

[     1329.1      519.58      1407.2      600.55         280]
[     827.53      637.68      929.76      768.65         278]
[     972.87      401.04      1026.2       462.8         273]
[     526.38      526.41      683.64      623.66         265]
[     994.96      462.64        1062      532.51         255]
[     1066.3      410.82      1123.1      462.01         250]
[     739.93      496.94      866.24      594.09         245]
[     807.16      787.36      936.48      941.31         234]
[     1047.2      532.36      1113.1      626.39         229]
[     21.721      635.22      185.71      756.57         227]
[     1428.3      476.06      1586.2      552.78         199]

0: 384x640 2 persons, 13 cars, 3 motorcycles, 2 trucks, 90.9ms
516 527 679 627
0.79
797 801 932 935
0.76
1437 477 1585 550
0.74
1608 771 1666 916
0.72
993 462 1062 536
0.68
732 494 864 595
0.62
124 735 240 861
0.61
1482 548 1516 637
0.52
825 642 924 777
0.52
1329 518 1407 601
0.48
2 639 176 757
0.44
1066 414 1123 46

0: 384x640 3 persons, 12 cars, 1 motorcycle, 3 trucks, 85.4ms
987 463 1057 544
0.8
1615 777 1677 926
0.79
769 855 907 940
0.72
475 528 647 633
0.69
800 669 910 812
0.67
696 507 835 602
0.61
1055 408 1118 460
0.59
1484 473 1584 539
0.59
1328 511 1413 592
0.54
1486 541 1517 632
0.54
273 670 441 827
0.44
972 411 1025 461
0.43
50 770 189 885
0.43
1431 483 1468 557
0.34
0 647 107 770
0.31
605 419 648 473
0.29
1042 544 1111 639
0.28
1280 446 1355 512
0.28
274 670 441 829
0.26
Speed: 2.0ms preprocess, 85.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[     1329.5      511.57      1411.4      592.12         280]
[     800.68      667.17       909.2      809.95         278]
[     972.55      410.73      1024.8       461.7         273]
[     477.07      529.28      645.77      631.39         265]
[     987.44      465.24      1057.1      541.86         255]
[     1056.2      408.43      1117.4      459.54         250]
[     1281.3      448.33      1351.6      511.49        

In [47]:
import cvzone
import math
model=YOLO("yolov8l.pt")
classNames =["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", 
             "fire nydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", 
             "cow", "elephant", "bear", "Zebra", "giraffe", "backpack", "umbrella", "handbag" "tie", 
             "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "haseball bat",
             
             "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass",
             "cup", "fork", "knife", "spoon", "bowt", 
             "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
             "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", 
             "diningtable", "toilet", "tvmonitor", "Laptop", "mouse", 
             "remote", "keyboard", "cell phone", "cellphone", "oven", 
             "toaster", "sink", "refrigeraton", "book", "clock", "vase", 
             "scissors", "teddy bear", "hair drier","toothbrush"]


tracker=Sort(max_age=20,min_hits=3,iou_threshold=0.3)
limits=[223,500,1473,500]
totalCount=[]

cap=cv2.VideoCapture('11.mp4')
cap.set(3,640)
cap.set(4,480)
mask=cv2.imread("mask_car.png")
while True:
    success,img=cap.read()
    mask_resized = cv2.resize(mask, (img.shape[1], img.shape[0]))  # Match width and height of img
    imgRegion = cv2.bitwise_and(img, mask_resized)

#     imgRegion=cv2.bitwise_and(img,mask)
    results=model(imgRegion,stream=True)
    detections=np.empty((0,5))
    for r in results:
        boxes=r.boxes
        for box in boxes:
            #bounding box
            
            x1,y1,x2,y2=box.xyxy[0]

            x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
#             cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
            w,h=x2-x1,y2-y1
#             bbox=int(x1),int(y1),int(w),int(h)
            
            
            
            print(x1,y1,x2,y2)

            cvzone.cornerRect(img,(x1,y1,w,h),l=15,colorR=(0, 0, 255),colorC=(0, 255, 255))
#             confidence
            conf=math.ceil((box.conf[0]*100))/100
                            
            print(conf)
            cls=int(box.cls[0])
            currentclass=classNames[cls]
            if currentclass=='car'or currentclass=='bus'or currentclass=='truck' or currentclass=='motorbiker' and conf>0.3:
                
#                 cvzone.putTextRect(img,f'{classNames[cls]} {conf}',(max(0,x1),max(15,y1)),scale=1,thickness=1,colorT=(0, 0, 255),
#                                colorR=(255, 0,0),offset=3)
                cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=1,colorR=(0, 0, 255),colorC=(0, 255, 255))
                currentArray=np.array([x1,y1,x2,y2,conf])
                detections=np.vstack((detections,currentArray))
                
                
    resultTracker=tracker.update(detections)
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,250),thickness=5)
    for result in resultTracker:
        x1,y1,x2,y2,id=result
        x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h=x2-x1,y2-y1
        
        cvzone.cornerRect(img,(x1,y1,w,h),l=15,rt=1,colorR=(100, 255, 255),colorC=(0, 150, 255))
        cvzone.putTextRect(img,f'{classNames[cls]} {conf}',(max(0,x1),max(35,y1)),scale=1,thickness=2,colorT=(0, 0, 255),
                               colorR=(255, 0,0),offset=3)
        cx,cy=x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
        
        if limits[0]<cx<limits[2] and limits[1]-20<cy<limits[1]+20:
            if totalCount.count(id)==0:
                
            
                totalCount.append(id)
    cvzone.putTextRect(img,f'COUNT:{len(totalCount)}',(50,50),scale=1,thickness=2,colorT=(0, 0, 255),
                               colorR=(255, 0,0),offset=3)
        
    cv2.imshow("images",img)
#     cv2.imshow("imagesRegion",imgRegion)
    cv2.waitKey(1)
        # Wait for 1ms, check if 'q' key is pressed
    key = cv2.waitKey(1)
    if key == 27:  # 'q' key to quit
        break


cap.release()
cv2.destroyAllWindows()



0: 640x384 7 persons, 1 car, 1 motorcycle, 1 truck, 558.5ms
112 815 186 977
0.86
414 915 509 1053
0.83
271 728 322 864
0.81
417 522 447 609
0.76
706 480 784 531
0.71
306 688 352 762
0.66
288 555 329 645
0.65
248 702 284 818
0.63
559 517 666 626
0.57
430 984 500 1085
0.38
Speed: 3.0ms preprocess, 558.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
[        559         517         666         626        1757]
[        706         480         784         531        1756]

0: 640x384 7 persons, 2 cars, 1 motorcycle, 682.3ms
112 817 185 983
0.86
423 901 517 1032
0.82
274 728 322 861
0.79
417 523 448 607
0.74
249 700 285 817
0.66
289 552 328 648
0.61
560 515 665 623
0.59
305 686 351 758
0.55
439 974 500 1079
0.55
706 480 784 527
0.52
Speed: 4.0ms preprocess, 682.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
[     559.88      515.12      665.12      622.88        1757]
[     706.73      479.55      783.27      527.45        1756]

0: 640x384 8 pers

576 480 670 565
0.84
156 1359 292 1531
0.8
99 841 179 1019
0.79
297 703 352 845
0.78
542 727 604 836
0.77
254 692 300 809
0.68
290 653 339 722
0.61
288 563 325 658
0.58
431 518 461 597
0.5
Speed: 2.0ms preprocess, 695.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
[     576.42      478.02      669.59      566.07        1757]

0: 640x384 8 persons, 1 car, 1 handbag, 534.6ms
96 843 176 1024
0.81
575 480 667 559
0.81
300 704 351 845
0.78
252 691 302 809
0.76
546 719 609 826
0.72
428 518 458 597
0.65
170 1339 301 1504
0.65
292 653 339 728
0.6
287 567 324 659
0.52
155 935 182 979
0.31
Speed: 1.9ms preprocess, 534.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
[     575.94      477.27      667.81      561.79        1757]

0: 640x384 8 persons, 1 car, 1 motorcycle, 556.2ms
95 849 177 1008
0.83
576 479 665 557
0.82
300 705 351 843
0.79
551 710 609 816
0.78
254 691 301 809
0.68
186 1318 307 1480
0.68
428 517 457 596
0.62
294 653 331 728
0.59
285 566 3

0: 640x384 9 persons, 1 handbag, 531.4ms
73 878 151 1047
0.83
0 1036 55 1253
0.78
300 674 349 804
0.77
406 496 436 589
0.75
223 671 272 791
0.73
591 584 641 674
0.63
341 1010 436 1205
0.56
250 562 293 658
0.45
138 971 165 1016
0.35
260 640 305 752
0.33
Speed: 2.0ms preprocess, 531.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 7 persons, 1 car, 1 fire hydrant, 1 handbag, 537.0ms
70 879 151 1042
0.86
0 1034 58 1242
0.77
299 673 346 799
0.77
405 496 436 589
0.75
224 673 271 789
0.74
592 579 641 672
0.71
257 641 303 711
0.44
137 971 164 1018
0.37
563 479 638 506
0.32
343 997 443 1193
0.31
Speed: 3.0ms preprocess, 537.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 10 persons, 1 handbag, 598.5ms
65 880 149 1050
0.87
0 1032 66 1248
0.81
403 497 435 591
0.76
592 575 641 659
0.76
221 672 269 789
0.75
297 671 345 800
0.74
348 989 440 1173
0.51
246 566 287 658
0.49
254 643 301 717
0.41
130 977 161 1020
0.33
254 643 303 761
0.31


[     1028.5      935.22      1077.7      1119.9        1759]

0: 640x384 6 persons, 1 car, 1 motorcycle, 1 handbag, 541.8ms
231 620 283 750
0.84
0 889 62 1073
0.84
357 759 427 901
0.75
163 616 218 714
0.62
313 479 345 543
0.54
164 655 215 747
0.48
46 992 70 1034
0.48
1023 941 1079 1128
0.44
136 541 183 647
0.4
Speed: 2.0ms preprocess, 541.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
[     1024.6      937.01      1077.6      1129.4        1759]

0: 640x384 5 persons, 1 motorcycle, 1 handbag, 705.7ms
227 618 277 748
0.83
0 889 57 1075
0.79
352 751 425 895
0.71
157 613 210 712
0.56
158 661 210 751
0.51
37 987 64 1038
0.43
309 479 340 542
0.38
Speed: 2.9ms preprocess, 705.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 6 persons, 1 car, 1 handbag, 642.6ms
219 610 269 738
0.82
0 887 53 1080
0.74
351 739 423 885
0.69
152 607 197 705
0.43
33 986 59 1033
0.37
123 539 169 645
0.36
1016 944 1079 1151
0.33
303 479 335 539
0.3
Speed: 2.0ms 